<a href="https://colab.research.google.com/github/tommybebe/til/blob/master/ml/Analyzing_a_financial_ML_model_deployed_on_Cloud_AI_Platform_with_the_What_if_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reference
- [Analyzing a financial ML model deployed on Cloud AI Platform with the What-if Tool
](https://codelabs.developers.google.com/codelabs/xgb-wit-caip/index.html)

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import collections
import witwidget

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

### 4. Download and process data

In [ ]:
!gsutil cp 'gs://mortgage_dataset_files/mortgage-small.csv' .

Copying gs://mortgage_dataset_files/mortgage-small.csv...
| [1 files][330.8 MiB/330.8 MiB]                                                
Operation completed over 1 objects/330.8 MiB.                                    


In [ ]:
COLUMN_NAMES = collections.OrderedDict({
    'as_of_year': np.int16,
    'agency_code': 'category',
    'loan_type': 'category',
    'property_type': 'category',
    'loan_purpose': 'category',
    'occupancy': np.int8,
    'loan_amt_thousands': np.float64,
    'preapproval': 'category',
    'county_code': np.float64,
    'applicant_income_thousands': np.float64,
    'purchaser_type': 'category',
    'hoepa_status': 'category',
    'lien_status': 'category',
    'population': np.float64,
    'ffiec_median_fam_income': np.float64,
    'tract_to_msa_income_pct': np.float64,
    'num_owner_occupied_units': np.float64,
    'num_1_to_4_family_units': np.float64,
    'approved': np.int8
})

In [ ]:
data = pd.read_csv(
    'mortgage-small.csv',
    index_col=False,
    dtype=COLUMN_NAMES
)
data = data.dropna()
data = shuffle(data, random_state=2)
data.head()

,as_of_year,agency_code,loan_type,property_type,loan_purpose,occupancy,loan_amt_thousands,preapproval,county_code,applicant_income_thousands,purchaser_type,hoepa_status,lien_status,population,ffiec_median_fam_income,tract_to_msa_income_pct,num_owner_occupied_units,num_1_to_4_family_units,approved
310650,2016,Consumer Financial Protection Bureau (CFPB),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Refinancing,1,110.0,Not applicable,119.0,55.0,Freddie Mac (FHLMC),Not a HOEPA loan,Secured by a first lien,5930.0,64100.0,98.81,1305.0,1631.0,1
630129,2016,Department of Housing and Urban Development (HUD),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Home purchase,1,480.0,Not applicable,33.0,270.0,Loan was not originated or was not sold in cal...,Not a HOEPA loan,Secured by a first lien,4791.0,90300.0,144.06,1420.0,1450.0,0
715484,2016,Federal Deposit Insurance Corporation (FDIC),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Refinancing,2,240.0,Not applicable,59.0,96.0,"Commercial bank, savings bank or savings assoc...",Not a HOEPA loan,Secured by a first lien,3439.0,105700.0,104.62,853.0,1076.0,1
887708,2016,Office of the Comptroller of the Currency (OCC),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Refinancing,1,76.0,Not applicable,65.0,85.0,Loan was not originated or was not sold in cal...,Not a HOEPA loan,Secured by a subordinate lien,3952.0,61300.0,90.93,1272.0,1666.0,1
719598,2016,National Credit Union Administration (NCUA),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Refinancing,1,100.0,Not applicable,127.0,70.0,Loan was not originated or was not sold in cal...,Not a HOEPA loan,Secured by a first lien,2422.0,46400.0,88.37,650.0,1006.0,1


In [ ]:
# Class labels - 0: denied, 1: approved
print(data['approved'].value_counts())

labels = data['approved'].values
data = data.drop(columns=['approved'])

1    665389
0    334610
Name: approved, dtype: int64


In [ ]:
dummy_columns = list(data.dtypes[data.dtypes == 'category'].index)
data = pd.get_dummies(data, columns=dummy_columns)

data.head()

,as_of_year,occupancy,loan_amt_thousands,county_code,applicant_income_thousands,population,ffiec_median_fam_income,tract_to_msa_income_pct,num_owner_occupied_units,num_1_to_4_family_units,...,"purchaser_type_Life insurance company, credit union, mortgage bank, or finance company",purchaser_type_Loan was not originated or was not sold in calendar year covered by register,purchaser_type_Other type of purchaser,purchaser_type_Private securitization,hoepa_status_HOEPA loan,hoepa_status_Not a HOEPA loan,lien_status_Not applicable (purchased loans),lien_status_Not secured by a lien,lien_status_Secured by a first lien,lien_status_Secured by a subordinate lien
310650,2016,1,110.0,119.0,55.0,5930.0,64100.0,98.81,1305.0,1631.0,...,0,0,0,0,0,1,0,0,1,0
630129,2016,1,480.0,33.0,270.0,4791.0,90300.0,144.06,1420.0,1450.0,...,0,1,0,0,0,1,0,0,1,0
715484,2016,2,240.0,59.0,96.0,3439.0,105700.0,104.62,853.0,1076.0,...,0,0,0,0,0,1,0,0,1,0
887708,2016,1,76.0,65.0,85.0,3952.0,61300.0,90.93,1272.0,1666.0,...,0,1,0,0,0,1,0,0,0,1
719598,2016,1,100.0,127.0,70.0,2422.0,46400.0,88.37,650.0,1006.0,...,0,1,0,0,0,1,0,0,1,0


In [ ]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

### 5. Build, train, and evaluate an XGBoost model

In [ ]:
model = xgb.XGBClassifier(
    objective='reg:logistic'
)

In [ ]:
model.fit(x_train, y_train)

XGBClassifier(objective='reg:logistic')

In [ ]:
y_pred = model.predict(x_test)
acc = accuracy_score(y_test, y_pred.round())
print(acc, '\n')

0.869992 



In [ ]:
model.save_model('model.bst')

### 6. Deploy model to Cloud AI Platform

#### Step 1: Create a Cloud Storage bucket for our model

In [ ]:
# Update these to your own GCP project, model, and version names
GCP_PROJECT = 'vibrant-airship-281001'
MODEL_BUCKET = 'gs://vibrant-airship-281001'
VERSION_NAME = 'v1'
MODEL_NAME = 'xgb_mortgage'

In [ ]:
!gsutil mb $MODEL_BUCKET

Creating gs://vibrant-airship-281001/...


#### Step 2: Copy the model file to Cloud Storage

In [ ]:
!gsutil cp ./model.bst $MODEL_BUCKET

Copying file://./model.bst [Content-Type=application/octet-stream]...
/ [1 files][ 49.1 KiB/ 49.1 KiB]                                                
Operation completed over 1 objects/49.1 KiB.                                     


In [ ]:
!gcloud config set project $GCP_PROJECT

Updated property [core/project].


#### Step 3: Create and deploy the model

In [ ]:
!gcloud ai-platform models create $MODEL_NAME

Using endpoint [https://ml.googleapis.com/]

Learn more about regional endpoints and see a list of available regions: https://cloud.google.com/ai-platform/prediction/docs/regional-endpoints
Created ml engine model [projects/vibrant-airship-281001/models/xgb_mortgage].


In [ ]:
!gcloud ai-platform versions create $VERSION_NAME \
--model=$MODEL_NAME \
--framework='XGBOOST' \
--runtime-version=1.15 \
--origin=$MODEL_BUCKET \
--python-version=3.7 \
--project=$GCP_PROJECT

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


#### Step 4: Test the deployed model

In [ ]:
%%writefile predictions.json
[2016.0, 1.0, 346.0, 27.0, 211.0, 4530.0, 86700.0, 132.13, 1289.0, 1408.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]

Writing predictions.json


In [ ]:
prediction = !gcloud ai-platform predict --model=xgb_mortgage --json-instances=predictions.json --version=$VERSION_NAME --verbosity=none

print(prediction) 

['Using endpoint [https://ml.googleapis.com/]', '[0.9997743964195251]']


### 7. Use the What-if Tool to interpret your model

In [ ]:
num_wit_examples = 500
test_examples = np.hstack((x_test[:num_wit_examples].values,y_test[:num_wit_examples].reshape(-1,1)))

In [ ]:
def adjust_prediction(pred):
  return [1 - pred, pred]

config_builder = (WitConfigBuilder(test_examples.tolist(), data.columns.tolist() + ['mortgage_status'])
  .set_ai_platform_model(GCP_PROJECT, MODEL_NAME, VERSION_NAME, adjust_prediction=adjust_prediction)
  .set_target_feature('mortgage_status')
  .set_label_vocab(['denied', 'approved']))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'classification', 'label_vocab': ['denied', 'approved'], 'feature_names': ['as…